# Extract icExpBackground  in  OGA for Holograms

 work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/02/04
- update : 2023/02/06


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_01/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g754a7f0350+935911ccad 	current w_2023_01 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import lsst.afw.math as afwMath
import lsst.daf.base as dafBase

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [7]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
transform = AsinhStretch() + PercentileInterval(99.)

In [9]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

## Configuration

In [10]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [11]:
DATE = 20230119

In [12]:
date_to_version_rebin1 = {
    20230118 : 'v13',
    20230117 : 'v14',
    20230119 : 'v15',
    20230131 : 'v16',
    20230201 : 'v17',
    20230202 : 'v18',
}

## Give the collection

In [13]:
#my_collection = "u/dagoret/BPS_test"
#my_collection = "u/dagoret/BPS_photimaging_202301"

#my_collection = "u/dagoret/BPS_test"
#my_collection = "u/dagoret/BPS_photimaging_202301"


#my_collection = "u/dagoret/BPS_manyspectro_v7" # january 20th 2023 (2023/01/17)
#my_collection = "u/dagoret/BPS_manyspectro_v8" # january 23th 2023 (2023/01/18)
#my_collection = "u/dagoret/BPS_manyspectro_v9" # january 23th 2023 (2023/01/19)
#my_collection = "u/dagoret/BPS_manyspectro_v10" # january 27th 2023 (2023/01/18), output with the _b, rebin=2
#my_collection = "u/dagoret/BPS_manyspectro_v11" # january 26th 2023 (2023/01/17), output with the _b, rebin=2
#my_collection = "u/dagoret/BPS_manyspectro_v12" # january 26th 2023 (2023/01/19), output with the _b, rebin=2,
#my_collection = "u/dagoret/BPS_manyspectro_v13" # january 31th 2023 (2023/01/18), rebin=1,
#my_collection = "u/dagoret/BPS_manyspectro_v14" # january 31th 2023 (2023/01/17), rebin=1,
#my_collection = "u/dagoret/BPS_manyspectro_v15" # january 31th 2023 (2023/01/18), rebin=1,
#my_collection = "u/dagoret/BPS_manyspectro_v16" # february 1st 2023 (2023/02/03), rebin=1,
#my_collection = "u/dagoret/BPS_manyspectro_v18" # february 2nd 2023 (2023/02/03), rebin=1,

the_version= date_to_version_rebin1[DATE] 
my_collection = f"u/dagoret/BPS_manyspectro_{the_version}"

In [14]:
file_icBgd_out = f"Table_icExpBackground_{DATE}.csv" 

## Config

In [15]:
# path index for each month
#DATE="20211103"
#filterdispersername = "empty~holo4_003"
#filterdispersername = "BG40~holo4_003"
#filterdispersername = "FELH0600~holo4_003"

# icExpBackground

In [16]:
datasetRefs = butler.registry.queryDatasets(datasetType='icExpBackground', collections=my_collection, where= "instrument='LATISS'")

In [17]:
df_icExpBg = pd.DataFrame(columns=['id','day_obs', 'seq_num','exposure','mjd','time','med','sig','N','band','physical_filter'])

In [18]:
all_ref_for_icExpBg  = []
all_dataId = []
all_exposures = []
all_icExpBg  = []
all_dateobs = []
all_mjd = []

for i, ref in enumerate(datasetRefs):
    print("========================datasetType = icExpBackground ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit...................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_visit = ref.dataId["visit"]
    the_exposure = the_visit
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0,}
    the_band=ref.dataId["band"]
    the_physical_filter = ref.dataId["physical_filter"]
    
    # extract time
    raw_img= butler.get('raw', dataId={'exposure': the_visit, 'instrument': 'LATISS', 'detector': 0}, collections = 'LATISS/raw/all')
    meta = raw_img.getMetadata()
    md = meta.toDict()
    date_obs = md['DATE-OBS']
    
    print(the_dataId)
    
    icExpBg = butler.get('icExpBackground', visit=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    all_ref_for_icExpBg.append(ref) 
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_icExpBg.append(icExpBg)
    
    outputStats = dafBase.PropertyList()

    # example of stat here : https://github.com/lsst/cp_pipe/blob/main/python/lsst/cp/pipe/cpFlatNormTask.py
    
    stats = afwMath.makeStatistics(icExpBg.getImage(),afwMath.MEANCLIP | afwMath.STDEVCLIP | afwMath.NPOINT)
    the_median = stats.getValue(afwMath.MEANCLIP)
    the_sigma = stats.getValue(afwMath.STDEVCLIP)
    the_N = stats.getValue(afwMath.NPOINT)
        
    print(the_median,the_sigma,the_N)
    
    the_visid_idx = i
    the_dateobs = pd.to_datetime(date_obs)
    the_mjd = Time(date_obs).mjd
    
    all_dataId.append(the_dataId) 
    all_dateobs.append(date_obs)
    all_mjd.append(the_mjd)
    
    #df_icExpBg = pd.DataFrame(columns=['id','day_obs', 'seq_num','exposure','mjd','time'])
    df_icExpBg.loc[i] = [i, the_day_obs, the_seq_num,the_exposure,the_mjd,the_dateobs,the_median,the_sigma,the_N,the_band,the_physical_filter]
                         
                        
    
    #registry_query = f"instrument='LATISS' and instrument=0 and visit={the_visit}]"
    #print(f"registry_query = {registry_query}")
    #info = registry.queryDimensionRecords('exposure',where = registry_query)
    
    #if i > 1:
    #    break

========================datasetType = icExpBackground ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2023011900392}
visit...................: 2023011900392
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/BPS_manyspectro_v15/20230130T220742Z
{'day_obs': 20230119, 'seq_num': 392, 'detector': 0}
28.30420845110118 8.60639054354903 16288000.0
========================datasetType = icExpBackground ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2023011900354}
visit...................: 2023011900354
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/BPS_manyspectro_v15/20230130T220742Z
{'day_obs': 20230119, 'seq_num': 354, 'detector': 0}
25.813

In [19]:
df_icExpBg

,id,day_obs,seq_num,exposure,mjd,time,med,sig,N,band,physical_filter
0,0,20230119,392,2023011900392,59964.204885,2023-01-20 04:55:02.076,28.304208,8.606391,16288000.0,white,empty~holo4_003
1,1,20230119,354,2023011900354,59964.186642,2023-01-20 04:28:45.868,25.813118,7.280438,16288000.0,white,empty~holo4_003
2,2,20230119,198,2023011900198,59964.061666,2023-01-20 01:28:47.949,29.181160,8.171530,16288000.0,white,empty~holo4_003
3,3,20230119,190,2023011900190,59964.057360,2023-01-20 01:22:35.897,32.579816,8.433680,16288000.0,white,empty~holo4_003
4,4,20230119,305,2023011900305,59964.162181,2023-01-20 03:53:32.402,26.977219,8.246312,16288000.0,white,empty~holo4_003
5,5,20230119,362,2023011900362,59964.189880,2023-01-20 04:33:25.662,26.924334,7.677623,16288000.0,white,empty~holo4_003
6,6,20230119,335,2023011900335,59964.177068,2023-01-20 04:14:58.695,33.282077,10.722654,16288000.0,white,empty~holo4_003
7,7,20230119,353,2023011900353,59964.186144,2023-01-20 04:28:02.850,26.269950,7.356869,16288000.0,white,empty~holo4_003
8,8,20230119,383,2023011900383,59964.200959,2023-01-20 04:49:22.868,33.016007,10.715849,16288000.0,white,empty~holo4_003
9,9,20230119,254,2023011900254,59964.122243,2023-01-20 02:56:01.803,46.803481,10.955340,16288000.0,white,empty~holo4_003


In [20]:
df_icExpBg.to_csv(file_icBgd_out)